In [1]:
%reload_ext autoreload
%autoreload 2

# IGLU/IGLU-PY results

In [2]:
import sys
from importlib.metadata import version

import iglu_py
import pandas as pd
import rpy2.robjects as ro

Error importing in API mode: ImportError("dlopen(/Users/staskh/Sandbox/iglu_python/.venv/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <38886600-97A2-37BA-9F86-5263C9A3CF6D> /Users/staskh/Sandbox/iglu_python/.venv/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/Users/staskh/Sandbox/iglu_python/.venv/lib/python3.11/site-packages/rpy2/rinterface/__init__.py:1185: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R

In [3]:
# Print versions for future references
print(f"Python version: {sys.version}")
print(f"R version: {ro.r('R.version.string')}")
iglu_version = str(ro.r('packageVersion("iglu")'))
print(f"iglu version: {iglu_version}")
print(f"iglu_py version: {version('iglu-py')}")
print(f"rpy2 version: {version('rpy2')}")

Python version: 3.11.10 (main, Oct  3 2024, 02:26:51) [Clang 14.0.6 ]
R version: [1] "R version 4.4.3 (2025-02-28)"

iglu version: [1] ‘4.2.2’

iglu_py version: 1.1.1
rpy2 version: 3.6.0


In [4]:
@iglu_py.bridge.df_conversion
def my_episode_calculation(data: pd.DataFrame, **kwargs):
    results = iglu_py.bridge.iglu_r.episode_calculation(data, **kwargs)

    if isinstance(results, pd.DataFrame):
        return results

    # here we have to handle NamedList
    assert isinstance(results,rpy2.rlike.container.NamedList)

    results_dict = {}

    for i, name in enumerate(results.names()):
        new_df = ro.conversion.rpy2py(results[i])
        new_df = new_df.replace({np.nan: None})  # To keep JSON happy
        new_df_dict = new_df.to_dict()
        if name in results_dict:
            results_dict[name].update(new_df_dict)
        else:
            results_dict[name] = new_df_dict
    return pd.DataFrame(results_dict['episodes']), pd.DataFrame(results_dict['data'])

In [7]:
test_data = pd.DataFrame(
    {
        "id": ["subject1"] * 16,
        "time": pd.date_range(start="2020-01-01 00:00:00", periods=16, freq="15min"),
        "gl": [
            100, 190, 190, 100, # hyperglycemia lv1
            100, 260, 260, 100, # hyperglycemia lv2
            100,  60,  60, 100,  # hypoglycemia lv1
            100,  50,  50, 100  # hypoglycemia lv2
            ],
    }
)

episodes = iglu_py.episode_calculation(test_data)
display(episodes)

,id,type,level,avg_ep_per_day,avg_ep_duration,avg_ep_gl,total_episodes
1,subject1,hypo,lv1,12.8,30.0,55.0,2.0
2,subject1,hypo,lv2,6.4,30.0,50.0,1.0
3,subject1,hypo,extended,0.0,0.0,NaN,0.0
4,subject1,hyper,lv1,12.8,30.0,225.0,2.0
5,subject1,hyper,lv2,6.4,30.0,260.0,1.0
6,subject1,hypo,lv1_excl,6.4,30.0,60.0,1.0
7,subject1,hyper,lv1_excl,6.4,30.0,190.0,1.0


In [8]:
episodes,data = iglu_py.episode_calculation(test_data,return_data=True)
display(episodes)
display(data)

,id,type,level,avg_ep_per_day,avg_ep_duration,avg_ep_gl,total_episodes
1,subject1,hypo,lv1,12.8,30.0,55.0,2.0
2,subject1,hypo,lv2,6.4,30.0,50.0,1.0
3,subject1,hypo,extended,0.0,0.0,NaN,0.0
4,subject1,hyper,lv1,12.8,30.0,225.0,2.0
5,subject1,hyper,lv2,6.4,30.0,260.0,1.0
6,subject1,hypo,lv1_excl,6.4,30.0,60.0,1.0
7,subject1,hyper,lv1_excl,6.4,30.0,190.0,1.0


,id,time,gl,segment,lv1_hypo,lv2_hypo,lv1_hyper,lv2_hyper,ext_hypo,lv1_hypo_excl,lv1_hyper_excl
1,subject1,2020-01-01 02:15:00+02:00,190.0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,subject1,2020-01-01 02:30:00+02:00,190.0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,subject1,2020-01-01 02:45:00+02:00,100.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,subject1,2020-01-01 03:00:00+02:00,100.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,subject1,2020-01-01 03:15:00+02:00,260.0,1,0.0,0.0,2.0,1.0,0.0,0.0,0.0
6,subject1,2020-01-01 03:30:00+02:00,260.0,1,0.0,0.0,2.0,1.0,0.0,0.0,0.0
7,subject1,2020-01-01 03:45:00+02:00,100.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,subject1,2020-01-01 04:00:00+02:00,100.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,subject1,2020-01-01 04:15:00+02:00,60.0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
10,subject1,2020-01-01 04:30:00+02:00,60.0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
